## Setup

```
> conda create -n genslate python=3.6
> conda activate genslate
> conda install pip
> conda install ipykernel
> python -m ipykernel install --user --name genslate --display-name "GenSlate"
> conda install pytorch==1.7.0 torchvision==0.8.0 cudatoolkit=9.2 -c pytorch
> conda install -c conda-forge scikit-learn
> conda isntall tqdm setproctitle
```

### Datasets and Simulation Environment

See corresponding [notebook](Dataset%20and%20Simulation.ipynb).

## 1. Experiment

General process:
* Real-world data: pretrain response model on entire set --> train generative model on training set --> evaluate on test set
* Simulation: generate dataset from simulator --> pretrain response model on train+val set --> train generative model on train set --> evaluate on test set

Basic commands:

DATASET COMMAND examples:

* Yoochoose:
```
> --dataset yoochoose --nouser
> --dataset movielens
> --dataset movielens --nouser
```

* Simulation:
```
> --dataset urm --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000
> --dataset urmp --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000
> --dataset urmpmr --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --mr_factor 0.2
```

TRAINING COMMAND examples:

```
> --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0 --nneg 100
```


### 1.1 Pretrain user response model from dataset

RESP MODEL COMMAND list:
* dim: embedding dimension size
* resp_struct: mlp structure as list of layer size

```
> python pretrain_env.py [DATASET COMMAND] [RESP MODEL COMMAND] [TRAINING COMMAND]
> python pretrain_env.py --dataset yoochoose --s 5 --nouser --dim 8 --resp_struct [40,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset movielens --s 5 --dim 8 --resp_struct [48,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset movielens --s 5 --nouser --dim 8 --resp_struct [40,256,256,5] --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset urmp --sim_dim 8 --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --pbias_min=-0.2 --pbias_max 0.2 --dim 8 --resp_struct [48,256,256,5] --epoch 10 --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
> python pretrain_env.py --dataset urmpmr --sim_dim 8 --n_user 1000 --n_item 3000 --n_train 100000 --n_val 10000 --n_test 10000 --pbias_min=-0.2 --pbias_max 0.2 --mr_factor 0.2 --dim 8 --resp_struct [48,256,256,5] --batch_size 64 --lr 0.001 --wdecay 0.001 --device cuda:0
```


### 1.2 Train generative model:

ENVIRONMENT COMMAND list:
* resp_path: saved response model path

MODEL COMMAND examples: (+8 for the first dim of each struct when --nouser flag is off)
```
> --model listcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --dec_struct [22,256,256,40] --prior_struct [6,128,128] --mask_train
> --model pivotcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
> --model pivotcvae_sgt_pi --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128] --mask_train
```

Format:
```
> python train_generative.py [DATASET COMMAND] [ENVIRONMENT COMMAND] [MODEL COMMAND] [TRAINING COMMAND] --beta 0.001
```

Example 1: 
* Environment: yoochoose
* Model: listcvae

```
> python train_generative.py --dataset yoochoose --nouser --resp_path resp/yoochoose_nouser/resp_[40,256,256,5]_dim8_BS64_lr0.00100_decay0.00100 --model listcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --dec_struct [22,256,256,40] --prior_struct [6,128,128] --mask_train --batch_size 64 --lr 0.0003 --wdecay 0.0 --device cuda:3 --nneg 1000 --beta 0.001
```



```
--resp_path resp/resp_[48,256,256,5]_yoochoose_nouser_BS64_dim8_lr0.00100_decay0.00010
```

MODEL COMMAND:

* ListCVAE
```
> --model listcvaewithprior --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --dec_struct [22,256,256,40] --prior_struct [6,128,128]
```

* PivotCVAE / PivotCVAEPrePermute
```
> --model pivotcvae --dim 8 --z_size 16 --s 5 --enc_struct [46,256,256] --psm_struct [22,256,256,8] --scm_struct [30,256,256,32] --prior_struct [6,128,128]
```

### 1.3 Beta search for a type of model: 

Use the same command in previous section, but set "--beta" to -1. This will iterate through the following beta values
> \[0.00001, 0.00003, 0.0001, 0.0003\] + 
> \[0.0005 + 0.0001 * i for i in range(5)\] +
> \[0.001 + 0.001 * i for i in range(10)\] +
> \[0.012 + 0.002 * i for i in range(10)\] + 
> \[0.1, 0.3, 1.0, 3.0, 10.0, 30.0\])

To change the beta list, check settings.py



### 1.4 Example

Yoochoose with ListCVAEWithPrior

In [14]:


def ms2str(ms):
    seconds=int(ms/1000)%60
    minutes=int(ms/(1000*60))%60
    hours=int(ms/(1000*60*60))%24
    days=int(ms/(1000*60*60*24))
    return str(days) + "d" + str(hours) + "h" + str(minutes) + "m" + str(seconds) + "s"
ms2str(360000000)
import torch
A = torch.randn(100,3000).to("cuda:1")
B = torch.randn(3000,200).to("cuda:1")
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
C = torch.mm(A,B)
end.record()
print(start.elapsed_time(end))

7.204864025115967


## Testing

### 2.1 Single Model

```
> python test_real.py --single_beta --dataset spotify --model_path model/listcvaewithprior_spotify_BS256_dim8_lr0.00030_decay0.00010_0.00001 --device cuda:2
```

### 2.2 Model with Different Beta

```
> python test_real.py --all_beta --dataset spotify --model_path model/listcvaewithprior_spotify_BS256_dim8_lr0.00030_decay0.00010 --device cuda:2
```

In [5]:
from data_extract import read_yoochoose, encode_yoochoose
train,val,test = read_yoochoose()
import numpy as np
print(len(np.unique(train["sessions"])))
# encode_yoochoose()

206036


In [3]:
print(train["features"])

[[   0    1    2    3    4]
 [   5    6    5    6    7]
 [   8    9   10   10   11]
 ...
 [6342 3358 6161 3358 2525]
 [5813 3945 3945 3945 3945]
 [2657  228  408  229  229]]


In [2]:
import data_extract as dae
from data_loader import UserSlateResponseDataset
from my_utils import make_model_path, make_result_path, Logger
import settings

import torch

modelPath = 'model/listcvaewithprior_spotify_BS256_dim8_lr0.00030_decay0.00010_0.00001'
# place to save test results
resultPath = 'results/spotify_listcvaewithprior_spotify_BS256_dim8_lr0.00030_decay0.00010_0.00001_singlebeta'
# logger
logger = Logger(resultPath)
# # test dataset
# if args.dataset == "spotify":
# datasets
slates, users, resps, train, val, test = dae.read_spotify()
#     testset = UserSlateResponseDataset(slates[test], users[test], resps[test], args.nouser)
testset = UserSlateResponseDataset(slates[test], users[test], resps[test], False)
# elif args.dataset == "yoochoose":
#     train, val, test = dae.read_yoochoose()
# #     testset = UserSlateResponseDataset(test["features"], test["sessions"], test["responses"], args.nouser)
#     testset = UserSlateResponseDataset(test["features"], test["sessions"], test["responses"], args.nouser)
# else:
#     raise NotImplemented


38056it [00:00, 191018.42it/s]

Log file path:
results/spotify_listcvaewithprior_spotify_BS256_dim8_lr0.00030_decay0.00010_0.00001_singlebeta++
Load data from "/home/sl1471/public/spotify/preprocessed/slates.csv"


17960960it [02:03, 145783.87it/s]
59736it [00:00, 595180.95it/s]

Load data from "/home/sl1471/public/spotify/preprocessed/users.csv"


17960960it [00:49, 363210.13it/s]
19096it [00:00, 190958.36it/s]

...

Load data from "/home/sl1471/public/spotify/preprocessed/val.csv"


1576877it [00:05, 307199.58it/s]
133414it [00:00, 645400.99it/s]

Load data from "/home/sl1471/public/spotify/preprocessed/test.csv"


1752925it [00:05, 343937.97it/s]


Initialize dataset
Slates shape: (1752925, 5)
Users shape: (1752925, 1)
Response shape: (1752925, 5)
Unique items: 545411
Unique users: 39
User embedding is used


In [3]:
def sample_users(environment, batch_size):
    up = torch.ones(environment.maxUserId + 1)
    sampledU = torch.multinomial(up, batch_size, replacement = True).reshape(-1).to(environment.device)
    return sampledU
    

In [4]:

def get_coverage(slates, N):
    """
    Item coverage for give slates
    @input:
     - slates: list of slates
     - N: the total number of items in D
    """
    return len(torch.unique(slates)) * 1.0 / N

def get_ILS(slates, embeds, normalize = False):
    """
    Intra-List Similarity, diversity can be calculated as (1 - ILS)
    @input:
     - slates: list of slates
     - embeds: nn.Embedding for all possible items
    """
    # obtain embeddings for all items
    emb = embeds(slates)
    # calculate similarities for each pair of items in each slate
    sims = torch.bmm(emb, emb.transpose(1,2)).reshape(slates.shape[0], -1)
    if normalize:
        sims /= torch.max(sims)
    # take the average for each slate
    sims = (torch.sum(sims, dim = 1) - slates.shape[1]) / (slates.shape[1] * (slates.shape[1] - 1))
    return sims

In [6]:
from tqdm import tqdm

def test_generative(environment, rec_model, logger, n_trail = 500, batch_size = 32, no_user = False, slate_size = 5):
    '''
    Generative performance:
    @report:
    - coverage: the percentage of candidate items that can be recommended
    - diversity: slate diversity given by 1 - ILS, where ILS is the intra-list similarity
    - enc: expected number of click by interacting with environment
    @input:
    - environment: the simulator (a.k.a. user response model)
    - rec_model: generative recommendation model of type f: (user,)context --> responses
    - logger: log file writer
    - n_trail: T, number of batch
    - batch_size: B, T*B is the total number of generated slates
    - no_user: set to True if the generative model takes user id as input
    - slate_size: size of slate L, used for setting up ideal responses as context
    '''
    logger.log("Test generative model on environment(user response model)")
    
    # generative performance
    
    diversities = torch.zeros(slate_size)
    allGenSlates = [torch.zeros((n_trail * batch_size, slate_size)) for i in range(slate_size)]
    expectedNClick = [[] for i in range(slate_size)]
    rec_model.set_candidate(False)
    
    with torch.no_grad():
        # repeat for several trails
        for k in tqdm(range(n_trail)):
            # sample users for each trail
            sampledUsers = sample_users(environment, batch_size)
            # test for different input condition/context
            context = torch.zeros(batch_size, slate_size).to(rec_model.device)
            for i in range(slate_size):
                # each time set one more target response from 0 to 1
                context[:,i] = 1
                # recommend should gives slate features of shape (B, L)
                if no_user:
                    rSlates, z = rec_model.recommend(context, return_item = True)
                    resp = environment(rSlates.view(batch_size, -1))
                else:
                    rSlates, z = rec_model.recommend(context, sampledUsers, return_item = True)
                    resp = environment(rSlates.view(batch_size, -1), sampledUsers)
                
                # diversity = 1 - ILS (intra list similarity)
                diversities[i] += torch.mean(-get_ILS(rSlates.view(batch_size, -1), rec_model.docEmbed).detach().cpu() + 1)
                # record the slate for calculating item coverage
                allGenSlates[i][k*batch_size: (k+1)*batch_size, :] = rSlates.detach().cpu()
                # the expected number of click
                expectedNClick[i].append(torch.sum(resp).detach().cpu().numpy())
        # final calculation of metrics
        coverages = []
        for i in range(slate_size):
            coverages.append(get_coverage(allGenSlates[i], len(rec_model.docEmbed.weight)))
        diversities = diversities / n_trail
        enc = np.mean(np.array(expectedNClick[i]), axis = 1)
    
    return {"coverage": coverage, "diversity": diversities, "enc": enc}

In [7]:
def standard_test(model_path, testset, resp_model, logger):
    '''
    Standard test consists:
    - a test on realworld dataset with generative and traditional performance metric
    - a test on generative performance on the pretrained simulator
    '''
    device = "cuda:2"
    recModel = torch.load(open(model_path, 'rb'))
    recModel.to(device)
    recModel.device = device
    resp_model.to(device)
    resp_model.device = device
#     reports[modelPath] = test_realworld(testset, recModel, logger)
    genReports[model_path] = test_generative(resp_model, recModel, logger)
    return {k:v for k,v in list(reports.items()) + list(genReports.items())}

In [8]:

# load environment (user response) model
respModel = torch.load(open('model/resp_[48,256,256,5]_spotify_BS64_dim8_lr0.00030_decay0.00010', 'rb'))


In [9]:
reports = standard_test(modelPath, testset, respModel, logger)

logger.log("Testset performance:")
logger.log(reports)

/home/sl1471/anaconda3/lib/python3.7/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'models.UserListCVAEWithPrior' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Test generative model on environment(user response model)


  0%|          | 0/500 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'view'

In [7]:
import torch
A = torch.randn(5,3)
b = torch.mean(torch.sum(A,1))
c = torch.zeros(3)
c[0] = b

In [8]:
c

tensor([-0.0602,  0.0000,  0.0000])